## scrap_list

Function to scrap a list of urls.<br>
dict_urls : dictionnary with urls(string) as key and is_house(bool) as value<br>
Returns a dictionnary compatible with the dataframe model<br>

## scrap

Function to scrap a specific url.<br>
url : url string <br>
is_house : bit indicating wheter the estate is a house or an appartment<br>
Returns a dictionnary with the propery name as key and it's value as value.<br>

    

In [74]:
from bs4 import BeautifulSoup 
import re 
import requests

def get_property_value(name):
    for elem in soup.find_all('tr'):
        if elem.th and re.search(name, str(elem.th.string)):
            return elem.td.contents[0].strip()


def scrap_list(dict_urls): 
    #listing all the property names
    properties = ["hyperlink" ,"locality", "postcode", "house_is", "property_subtype",	"price", "sale", "rooms_number", "area", "kitchen_has", "furnished",	"open_fire", "terrace", "terrace_area", "garden", "garden_area", "land_surface", "land_plot_surface", "facades_number", "swimming_pool_has"]

    #making a dict with all the property names as key and an empty list as value
    for property_name in properties:
        dict_dataframe[property_name] = []

    #scrap each url of the input and put the result into a variable
    for key in dict_urls: 
        dict_result_scrapping = scrap(key, dict_urls[key])

        #for each property (key) of the scrapping out put, match it with dataframe property. If none exist, just use None
        for key1 in dict_dataframe:
            dict_dataframe[key1].append(dict_result_scrapping.get(key1, False) or None)            
    
    return dict_dataframe

def scrap(url, is_house): 
    dict = {} 
    r = requests.get(url) 
    soup = BeautifulSoup(r.content,'html.parser') 
    
    dict["hyperlink"] = url
    dict["locality"] = url.split("/")[7]
    dict["postcode"] = url.split("/")[8]
    dict['house_is'] = is_house
    dict['property_subtype'] = url.split("/")[5]
    dict['price'] = ''
    dict['sale'] = ''
    dict['rooms_number'] = ''
    dict['kitchen_has'] = ''
    dict['furnished'] = ''
    dict['open_fire'] = ''
    dict['terrace'] = ''
    dict['terrace_area'] = ''
    dict['garden'] = ''
    dict['garden_area'] = ''
    dict['land_surface'] = ''
    dict['land_plot_surface'] = ''
    dict['facades_number'] = ''
    dict['swimming_pool_has'] = ''
    
    return dict

print(get_property_value("Bedrooms"))

2


In [71]:

from bs4 import BeautifulSoup
import requests
import re

r = requests.get(r"https://www.immoweb.be/en/classified/apartment/for-sale/auderghem/1160/8899851?searchId=5f6c49990b1b6") 
soup = BeautifulSoup(r.content,'html.parser') 




           
    # if elem.find_all('th', text = re.compile("Living area")). is not None:
    #     print(elem.td.string)


    

87
